## Data generation And Restocking with model predictions

In [36]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
from pprint import pprint
import copy

stock_file_path = './initial_stock.csv'
classified_products_file_path = './products.csv'

stock_df = pd.read_csv(stock_file_path)
classified_products_df = pd.read_csv(classified_products_file_path)

In [40]:
print(stock_df.head())

   product_id  quantity expiry_date  shelf_life  time_in_stock
0           1        21  2025-02-28          47             13
1           2        15  2025-05-13         121             20
2           3        25  2025-05-02         110              8
3           4        26  2025-06-17         156              0
4           5        23  2025-08-29         229             16


In [42]:
print(classified_products_df.head())

         Product_Name        Brand    Category Usage_Frequency  Price_USD  \
0      Magic Eyeliner     Shiseido    CC Cream          Weekly     149.16   
1   Perfect Lip Liner  Huda Beauty   Lip Gloss          Weekly      62.28   
2  Divine Highlighter     Bourjois     Mascara      Occasional      51.65   
3   Ultra Highlighter     Shiseido      Primer           Daily      71.25   
4     Ultra Lip Gloss      Kiehl’s  Eye Shadow          Weekly      78.36   

   Rating  Number_of_Reviews Product_Size  Skin_Type Gender_Target  \
0     4.4               6158         50ml  Sensitive        Female   
1     3.7               7809        150ml        Dry          Male   
2     2.1               3815        250ml        Dry        Female   
3     4.0               5749        150ml       Oily        Female   
4     1.9               3534         30ml     Normal          Male   

  Packaging_Type Main_Ingredient  Cruelty_Free Country_of_Origin Product Name  \
0            Jar     Shea Butter   

## Trained model

In [55]:
import joblib

best_model = joblib.load("../model/xgboost_model_first.pkl")

print("Modelo cargado correctamente")

Modelo cargado correctamente


### Predecimos compras enero de 2025

In [56]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

data = [
    {"product_id": 1, "year": 2024, "month": 12, "remaining_stock": 150},
    {"product_id": 1, "year": 2025, "month": 2, "remaining_stock": 140},
    {"product_id": 1, "year": 2025, "month": 3, "remaining_stock": 130},
    {"product_id": 1, "year": 2025, "month": 4, "remaining_stock": 120},
    {"product_id": 1, "year": 2025, "month": 5, "remaining_stock": 110},
    {"product_id": 1, "year": 2025, "month": 6, "remaining_stock": 100},
    {"product_id": 1, "year": 2025, "month": 7, "remaining_stock": 90},
    {"product_id": 1, "year": 2025, "month": 8, "remaining_stock": 80},
    {"product_id": 1, "year": 2025, "month": 9, "remaining_stock": 70},
    {"product_id": 1, "year": 2025, "month": 10, "remaining_stock": 60},
    {"product_id": 1, "year": 2025, "month": 11, "remaining_stock": 50},
    {"product_id": 1, "year": 2025, "month": 12, "remaining_stock": 40},
]


sales_test_month_year = pd.DataFrame(data)


new_data = pd.DataFrame({
    'product_id': sales_test_month_year["product_id"],
    'year': sales_test_month_year["year"],
    'month': sales_test_month_year["month"],
    'remaining_stock':sales_test_month_year["remaining_stock"]
})


new_predictions = best_model.predict(new_data)
sales_test_month_year["predictions"] = new_predictions
sales_test_month_year["predictions"] = sales_test_month_year["predictions"].astype(int)
sales_test_month_year

,product_id,year,month,remaining_stock,predictions
0,1,2024,12,150,29
1,1,2025,2,140,18
2,1,2025,3,130,18
3,1,2025,4,120,17
4,1,2025,5,110,17
5,1,2025,6,100,16
6,1,2025,7,90,24
7,1,2025,8,80,24
8,1,2025,9,70,18
9,1,2025,10,60,18


In [57]:
sales_test_file_path = './sales_test.csv'
sales_test = pd.read_csv(sales_test_file_path)

print(sales_test.head())

print(sales_test.dtypes)

if 'date' in sales_test.columns:
    sales_test['date'] = pd.to_datetime(sales_test['date'], errors='coerce')  # Asegura que las fechas se conviertan correctamente

filtered_sales = sales_test[(sales_test['date'].dt.year == 2024) & (sales_test['date'].dt.month == 12) & (sales_test['product_id'] == 1)]

first_record_december = filtered_sales.head(1)

print(first_record_december)


   sale_id  product_id        date  quantity  total_price batch_expiry_date  \
0        1           8  2020-01-01         1        89.35        2025-02-22   
1        2           6  2020-01-01         1       102.88        2025-02-17   
2        3           3  2020-01-01         1        51.65        2025-03-30   
3        4           7  2020-01-01         1        45.24        2025-03-15   
4        5           9  2020-01-01         1       115.69        2025-05-25   

   remaining_stock  warehouse_occupancy_rate  
0               33                      85.4  
1               21                      85.2  
2               36                      85.0  
3               51                      84.8  
4               13                      84.6  
sale_id                       int64
product_id                    int64
date                         object
quantity                      int64
total_price                 float64
batch_expiry_date            object
remaining_stock            

In [65]:
print(first_record_december["remaining_stock"].iloc[0])

data = [
    {"product_id": 1, "year": 2025, "month": 1, "remaining_stock": first_record_december["remaining_stock"].iloc[0]},
    {"product_id": 1, "year": 2025, "month": 2, "remaining_stock": first_record_december["remaining_stock"].iloc[0]}
]

sales_test_month_year = pd.DataFrame(data)


new_data = pd.DataFrame({
    'product_id': sales_test_month_year["product_id"],
    'year': sales_test_month_year["year"],
    'month': sales_test_month_year["month"],
    'remaining_stock':sales_test_month_year["remaining_stock"]
})


new_predictions = best_model.predict(new_data)
sales_test_month_year["predictions"] = new_predictions
sales_test_month_year["predictions"] = sales_test_month_year["predictions"].astype(int)
sales_test_month_year

24


,product_id,year,month,remaining_stock,predictions
0,1,2025,1,24,34
1,1,2025,2,24,20


In [47]:
start_date = datetime(2025, 1, 1)
end_date = datetime(2025, 12, 31)
current_date = start_date
warehouse_capacity = 500

products = pd.merge(stock_df, classified_products_df, left_on='product_id', right_on='id')

initial_stock = {}
for _, row in stock_df.iterrows():
    product_id = row['product_id']
    batch = {
        'quantity': row['quantity'],
        'expiry_date': datetime.strptime(row['expiry_date'], '%Y-%m-%d')
    }
    if product_id not in initial_stock:
        initial_stock[product_id] = []
    initial_stock[product_id].append(batch)

stock = copy.deepcopy(initial_stock)

In [67]:
sales_data = []  
unsold_products = {}  

def restock(stock, initial_stock, current_date):

    print('RESTORIIIIINGGG!!!')

    for product_id, batches in stock.items():
        initial_quantity = sum(batch['quantity'] for batch in initial_stock.get(product_id, []))
        
        current_quantity = sum(batch['quantity'] for batch in batches)
        
        missing_quantity = initial_quantity - current_quantity
        
        if missing_quantity > 0:

            
            data = [
    {"product_id":product_id, "year": 2025, "month": current_date.month, "remaining_stock": 24},
]

            sales_test_month_year = pd.DataFrame(data)


            new_data = pd.DataFrame({
                'product_id': sales_test_month_year["product_id"],
                'year': sales_test_month_year["year"],
                'month': sales_test_month_year["month"],
                'remaining_stock':sales_test_month_year["remaining_stock"]
            })


            new_predictions = best_model.predict(new_data)
            sales_test_month_year["predictions"] = new_predictions

            sales_test_month_year["predictions"] = sales_test_month_year["predictions"].astype(int)
            missing_quantity = sales_test_month_year["predictions"]

            new_batch = {
                'quantity': missing_quantity,
                'expiry_date': current_date + timedelta(days=random.randint(180, 240))
            }
            stock[product_id].append(new_batch)



# Factores de ajuste
def get_adjustment_factor(current_date):

    # Ajustes por día del mes (principios de mes)
    if current_date.day <= 7:
        day_factor = 1.3
    else:
        day_factor = 1

    # Ajustes por día de la semana (fines de semana)
    if current_date.weekday() in [5, 6]:  # Sábado y domingo
        weekend_factor = 3.0
    else:
        weekend_factor = 1

    # Factor final
    return day_factor * weekend_factor

monthly_demand_factor = {
    1: 1.5,  # Enero
    7: 1.3,  # Julio
    8: 1.3,  # Agosto
    12: 1.5, # Diciembre
    2: 1,
    3: 1,
    4: 1,
    5: 1,
    6: 1,
    9: 1,
    10: 1,
    11: 1
}

# Simulate daily sales
while current_date <= end_date:
    month = current_date.month
    
    # Base sales adjusted by month and day factors
    base_sales = random.randint(4,8)
    adjustment_factor = get_adjustment_factor(current_date)
    daily_sales_count = int(base_sales * adjustment_factor) * monthly_demand_factor[month]
    
    daily_sales = []
    total_items_sold = 0

    # Al inicio del día:
    # Verificar productos caducados
    for product_id, batches in stock.items():
        expired_batches = [batch for batch in batches if batch['expiry_date'] <= current_date]
        if expired_batches:
            # Registrar los lotes caducados en `unsold_products`
            if product_id not in unsold_products:
                unsold_products[product_id] = []
            unsold_products[product_id].extend(expired_batches)
            
            # Eliminar los lotes caducados del stock
            stock[product_id] = [batch for batch in batches if batch['expiry_date'] > current_date]

    # Reabastecer si es el primer lunes del mes
    if current_date.weekday() == 0 and (1 <= current_date.day <= 7 or 15 <= current_date.day <= 21):
        restock(stock, initial_stock, current_date)

    
    
    while total_items_sold < daily_sales_count:
        
        # Extract available products [6,8,28,23,1,14,...]
        available_products = [p for p in stock if any(batch['quantity'] > 0 for batch in stock[p])]
        """ print('Available products: ', available_products) """

    
        # Si no hay productos en el almacén
        if not available_products:
            raise ValueError("ALMACÉN VACÍO!!!!!")
            break
        

        # Clasificar productos según la rotación
        high_rotation = [p for p in available_products if products[products['product_id'] == p].iloc[0]['Rotation'] == 'High']
        medium_rotation = [p for p in available_products if products[products['product_id'] == p].iloc[0]['Rotation'] == 'Medium']

        # 1. Seleccionar 5 productos de media rotación
        medium_pool = random.sample(medium_rotation, min(5, len(medium_rotation)))

        # 2. Modificar el pool de alta rotación
        high_pool = high_rotation.copy()
        
        # 3. Sumar los pools de alta y media rotación. De la suma total quito 2 y sumo entre 0 y 2.
        final_pool = medium_pool + high_pool

        if len(final_pool) > 2:
            final_pool = random.sample(final_pool, len(final_pool) - 2)  # Quitar 2 productos aleatorios
            duplicates = random.choices(final_pool, k=random.randint(0, 2)) # Seleccionar 2 productos para duplicar
            final_pool.extend(duplicates)  # Duplicar productos seleccionados

        # 3. Combinar ambos pools y mezclarlos
        final_pool = medium_pool + high_pool
        random.shuffle(final_pool)  # Mezclar los productos

        # 4. Procesar ventas en el orden del pool mezclado
        for product_id in final_pool:
            if total_items_sold >= daily_sales_count:
                break

            # Obtener detalles del producto
            product_details = products[products['product_id'] == product_id].iloc[0]

            # Determinar la cantidad a vender (1 o 2 unidades con proporción 90-10)
            sold_quantity = 1 if random.random() < 0.9 else 2

            # Verificar el stock disponible
            batches = sorted(stock[product_id], key=lambda batch: batch['expiry_date'])
            total_stock = sum(b['quantity'] for b in batches)
            sold_quantity = min(sold_quantity, total_stock)

            # Procesar ventas desde los lotes
            remaining_quantity = sold_quantity
            for batch in batches:
                if remaining_quantity <= 0:
                    break
                quantity_to_sell = min(batch['quantity'], remaining_quantity)
                batch['quantity'] -= quantity_to_sell
                remaining_quantity -= quantity_to_sell

            # Registrar la venta
            total_items_in_stock = sum(
                batch['quantity'] for product_batches in stock.values() for batch in product_batches
            )
            sales_data.append({
                'sale_id': len(sales_data) + 1,
                'product_id': product_id,
                'date': current_date,
                'quantity': sold_quantity,
                'total_price': sold_quantity * product_details['Price_USD'],
                'batch_expiry_date': batches[0]['expiry_date'],  # Lote más cercano a caducar
                'remaining_stock': total_stock - sold_quantity,
                'warehouse_occupancy_rate': total_items_in_stock / warehouse_capacity * 100
            })

            total_items_sold += sold_quantity

            # Eliminar lotes vacíos
            batches = [batch for batch in batches if batch['quantity'] > 0]
            stock[product_id] = batches

            # Calcular el total de ítems en el almacén
            print(f"Total de ítems en el almacén: {total_items_in_stock}")


        
    print(f"\nCurrent_date: {current_date}\n")
    #pprint(sales_data)
    # Move to the next day
    current_date += timedelta(days=1)

# Convert sales data to a DataFrame and save
sales_df = pd.DataFrame(sales_data)
sales_file_path = './sales_test.csv'
sales_df.to_csv(sales_file_path, index=False)

RESTORIIIIINGGG!!!


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().